In [4]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import librosa.display

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score


In [54]:
import numpy as np
import librosa

def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, duration=30)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc_delta = librosa.feature.delta(mfcc)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        rms = librosa.feature.rms(y=y)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        features = np.hstack([
            np.mean(mfcc.T, axis=0),
            np.mean(mfcc_delta.T, axis=0),
            np.mean(chroma.T, axis=0),
            np.mean(contrast.T, axis=0),
            np.mean(rolloff.T, axis=0),
            np.mean(zcr.T, axis=0),
            np.mean(rms.T, axis=0),
            tempo  
        ])

        return features

    except Exception as e:
        print(f"Failed to extract from {file_path}: {e}")
        return None


In [60]:
features=[]
labels=[]
GENRE_DIR=r"Data\genres_original"

for genre in os.listdir(GENRE_DIR):
    genre_folder=os.path.join(GENRE_DIR,genre)
    for file in os.listdir(genre_folder):
        try:
            file_path=os.path.join(genre_folder,file)
            audio_features=extract_features(file_path)
            features.append(audio_features)
            labels.append(genre)
        except Exception as e:
            print(f"Error with {file}:{e}")
            


jazz:  54%|████████████████████████████████████████▌                                  | 54/100 [00:44<00:37,  1.22it/s]C:\Users\Manan Sharma\AppData\Local\Temp\ipykernel_20236\3882780587.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, duration=30)
E:\ML Projects\Genre-Classifier-KNN\venv\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Failed to extract from Data\genres_original\jazz\jazz.00054.wav: 



Genres: 100%|██████████████████████████████████████████████████████████████████████████| 10/10 [13:29<00:00, 80.92s/it]


In [20]:
import os

GENRE_DIR = r"Data\genres_original"

# Check if path exists
print("Genres:", os.listdir(GENRE_DIR))  # Should print: ['blues', 'classical', 'rock', ...]


Genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [75]:
X=np.array(features)
y=np.array(labels)

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y_encoded=label_encoder.fit_transform(y)

X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,stratify=y_encoded,random_state=42)

scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)


In [76]:
knn=KNeighborsClassifier(n_neighbors=5,metric='euclidean')
knn.fit(X_train_scaled,y_train)

y_pred=knn.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.505

Classification Report:
               precision    recall  f1-score   support

       blues       0.50      0.40      0.44        20
   classical       0.76      0.80      0.78        20
     country       0.38      0.45      0.41        20
       disco       0.31      0.45      0.37        20
      hiphop       0.54      0.35      0.42        20
        jazz       0.42      0.80      0.55        20
       metal       0.67      0.80      0.73        20
         pop       0.62      0.65      0.63        20
      reggae       0.44      0.20      0.28        20
        rock       0.60      0.15      0.24        20

    accuracy                           0.51       200
   macro avg       0.52      0.51      0.49       200
weighted avg       0.52      0.51      0.49       200



In [64]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_neighbors': [3, 5, 7, 9],
    'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), params, cv=5)
grid.fit(X_train_scaled, y_train)

print("Best KNN params:", grid.best_params_)
print("Best CV score:", grid.best_score_)

# Evaluate on test set
y_pred = grid.predict(X_test_scaled)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


Best KNN params: {'metric': 'manhattan', 'n_neighbors': 9}
Best CV score: 0.5931996855345912
Test Accuracy: 0.555
